# Backtest Analysis

In [ ]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

import matplotlib as mpl
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

%matplotlib widget
# inline is the old interface

In [ ]:
ticker = 'ANET'

In [ ]:
# import pandas_datareader as pdr
# import os
# from datetime import date 

# if not os.path.exists(os.path.join(r'../md/daily',f'{ticker}.csv')):
#     df = pdr.DataReader(ticker, 'yahoo', start='2012-01-01', end=date.today().isoformat())
#     df.to_csv(os.path.join(r'../md/daily',f'{ticker}.csv'))

### Run backtest

In [ ]:
import backfire as bf

s = bf.BuyAndHoldStrategy(
    env=bf.Environment(md=r"../md/daily", out_dir=r"./out_test")) 

s.backtest(ticker=ticker, from_date="2015-01-01")

label = "SDS_signal=BuyAndHold_sl=1.0_tp=1000_pos=fixed"
print("backtest done")

In [ ]:
import backfire as bf

env = bf.Environment(md=r"../md/daily", out_dir=r"./out_test")
signal = bf.Stage2Signal()
# set very large tp effectively disabling it
s = bf.SignalDrivenStrategy(
    env=env,
    signal=signal,
    sl=0.07, tp=5.0)

s.backtest(ticker=ticker, from_date="2015-01-01")

print("backtest done")

In [ ]:
label = "SDS_signal=flatbase20_0.10_sl=0.07_tp=0.2_pos=fixed"
label = "SDS_signal=Stage2_rise200dlength=1_rise200dstep=0.01_sl=0.07_tp=5.0_pos=fixed"

### Visualize signals and positions

In [ ]:
df = pd.read_csv(os.path.join(r"./out_test", f"pos_{ticker}_{label}.csv"),index_col=0,parse_dates=True)
df['50dSMA'] = df.C.rolling(50, min_periods=20).mean()
df['200dSMA'] = df.C.rolling(200,min_periods=100).mean()

In [ ]:
# from_to = ['2020-05-01', '2021-03-01']
# from_to = [datetime.strptime(x, "%Y-%m-%d").date() for x in from_to]
# df = df.loc[from_to[0]:from_to[1]]

In [ ]:
fig = make_subplots(specs=[[{"secondary_y":True}]])

fig.add_trace(go.Ohlc(
    name="price",
    x=df.index, open=df.O, high=df.H, low=df.L, close=df.C,
    xhoverformat='%Y-%m-%d'),secondary_y = False)
fig.add_trace(go.Scatter(name="ma50",x=df.index,y=df["50dSMA"],line={'color': 'red', 'width': 1}),secondary_y=False)
fig.add_trace(go.Scatter(name="ma200",x=df.index,y=df["200dSMA"],line={'color': 'black', 'width': 1}),secondary_y=False)

# fig.add_trace(go.Scatter(x = df.index,y = df.pos,name="pos", line_color = 'grey',mode='lines',
#                          line={'dash':'dash'},marker_line_width=1,opacity=0),secondary_y = True)

fig.add_trace(go.Scatter(name=signal.name,x=df.index,y=df.es,line_color='cyan',mode='lines',line={'dash':'dash'},marker_line_width=1,opacity=0.4),secondary_y = False)


fig.update_layout(autosize=False, 
                  width=1500,
                  height=800,
                  title="",
                  yaxis_title="Price",
                  xaxis_rangeslider_visible=False,
                  showlegend=False,
                 hovermode='x unified')
fig.show()

### Trades

In [ ]:
trades = pd.read_csv(os.path.join(r'./out_test',f'trades_{ticker}_{label}.csv'),index_col=0,header=0)
trades

### Stats

In [ ]:
trades = pd.read_csv(os.path.join(r'./out_test',f'stats_{ticker}_{label}.csv'),index_col=0,header=0)
trades